# Class mapping exercise

#### Load python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

### Election map
#### What percentage of vote did Biden/Trump receive in California counties?

In [4]:
# Values: https://raw.githubusercontent.com/stiles/notebooks/master/elections/data/ca_counties_election_results_2020.csv
# Geo: https://raw.githubusercontent.com/stiles/usc/main/data/geo/counties_simple.geojson

#### Read election results

In [5]:
election_df = pd.read_csv(
    "https://raw.githubusercontent.com/stiles/notebooks/master/elections/data/ca_counties_election_results_2020.csv",
    dtype={"county_fips": str},
)

In [6]:
election_df.head()

,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff,winner
0,California,001,Alameda,136309,617659,769864,-481350,0.177056,0.802296,0.625240,Biden
1,California,003,Alpine,244,476,741,-232,0.329285,0.642375,0.313090,Biden
2,California,005,Amador,13585,8153,22302,5432,0.609138,0.365573,0.243566,Trump
3,California,007,Butte,48730,50426,102042,-1696,0.477548,0.494169,0.016621,Biden
4,California,009,Calaveras,16518,10046,27164,6472,0.608084,0.369828,0.238257,Trump


#### Read county boundaries

In [7]:
ca_counties_gdf = gpd.read_file(
    "https://raw.githubusercontent.com/stiles/usc/main/data/geo/counties_simple.geojson",
    dtype={"county_fips": str},
)

In [8]:
ca_counties_gdf.head()

,county_name,county_fips,geometry
0,Alameda,001,"POLYGON ((-122.27125 37.90503, -122.27024 37.9..."
1,Alpine,003,"POLYGON ((-119.58667 38.71420, -119.58653 38.7..."
2,Amador,005,"POLYGON ((-120.07246 38.70276, -120.07249 38.6..."
3,Butte,007,"POLYGON ((-121.07661 39.59729, -121.07945 39.5..."
4,Calaveras,009,"POLYGON ((-120.01792 38.43586, -120.01788 38.4..."


#### Merge the two 

In [9]:
ca_counties_prez20 = pd.merge(
    ca_counties_gdf, election_df, on=["county_fips", "county_name"]
)

In [10]:
ca_counties_prez20.to_file(
    "../data/geo/president_county_results_2020.geojson", driver="GeoJSON"
)

---

### Vehicle map
##### What's the Tesla ownership rate by LA ZIP Codes?

In [11]:
# Values: https://raw.githubusercontent.com/stiles/notebooks/master/california-fuel-vehicles/output/tesla_zips_slim.csv
# Geo: https://raw.githubusercontent.com/stiles/notebooks/master/zips/data/processed/ca_zips_simple.geojson.json

#### Read Tesla data

In [12]:
tesla_df = pd.read_csv(
    "https://raw.githubusercontent.com/stiles/notebooks/master/california-fuel-vehicles/output/tesla_zips_slim.csv",
    dtype={"zip": str},
)

In [13]:
tesla_df.head()

,zip,name,teslas,all_vehicles,tesla_rate_1k,avg_house_income,avg_net_worth
0,90004,Los Angeles,200,22734,8.80,65893,232653
1,90005,Los Angeles,58,12518,4.63,53191,105569
2,90006,Los Angeles,26,17324,1.50,42242,39289
3,90007,Los Angeles,27,10707,2.52,38564,39966
4,90008,Los Angeles,54,12611,4.28,56286,284531


#### Read ZIP Codes geo file

In [14]:
zips_la_gdf = gpd.read_file(
    "https://raw.githubusercontent.com/stiles/notebooks/master/zips/data/processed/ca_zips.geojson"
)

In [15]:
zips_la_gdf.head()

,id,name,county_name,geometry
0,90001,Los Angeles,Los Angeles County,"POLYGON ((-118.24329 33.98901, -118.24754 33.9..."
1,90002,Los Angeles,Los Angeles County,"POLYGON ((-118.23085 33.95772, -118.23108 33.9..."
2,90003,Los Angeles,Los Angeles County,"MULTIPOLYGON (((-118.28268 33.98926, -118.2826..."
3,90004,Los Angeles,Los Angeles County,"MULTIPOLYGON (((-118.33852 34.06891, -118.3371..."
4,90005,Los Angeles,Los Angeles County,"POLYGON ((-118.31879 34.05514, -118.31852 34.0..."


#### Merge the two

In [16]:
tesla_zips_gdf = pd.merge(
    zips_la_gdf, tesla_df, left_on="id", right_on="zip", how="left"
).fillna(0)

#### Export merged geo file

In [17]:
tesla_zips_gdf.to_file(
    "../data/geo/la_county_zips_tesla_rate.geojson", driver="GeoJSON"
)

---

### Population map
##### How's the white population distributed in LA by neighborhood? 

In [18]:
# Values:https://raw.githubusercontent.com/stiles/usc/main/data/processed/la_neighborhood_demographics.csv
# Geo: https://raw.githubusercontent.com/stiles/usc/main/data/geo/la-neighborhoods.geojson

#### Read race/ethnicity data

In [19]:
whites_la_df = pd.read_csv(
    "https://raw.githubusercontent.com/stiles/usc/main/data/processed/la_neighborhood_demographics.csv"
)

In [20]:
whites_la_df.head()

,name,total_population,white_alone,black_alone,american_indian_and_alaska_native,asian_alone,native_hawaiian_and_pacific_islander,other_alone,two_or_more_races,latino_alone,asians_all,other_all,white_percent,black_percent,american_indian_percent,asian_percent,native_hawaiian_percent,other_percent,two_or_more_races_percent,latino_percent,asians_all_percent,other_all_percent,nonwhite_percent,majority_nonwhite,majority_white,majority_black,majority_american_indian_and_alaska_native,majority_asian,majority_native_hawaiian,majority_other,majority_two_or_more_races,majority_latino,median_householdincome,total_occupied_housing_units,owner_occupied,renter_occupied,owner_percent,renter_percent,majority_owner,majority_renter,majorityhigherthanmedianincomeLA
0,Acton,13533.0,3029.552482,43.082892,9.114016,122.284944,0.280278,0.168167,64.192933,1018.768948,122.565222,73.475116,0.223864,0.003184,0.000673,0.009036,0.000021,0.000012,0.004743,0.075280,0.009057,0.005429,0.776136,True,False,False,False,False,False,False,False,False,99453.506605,1474.265218,1329.164851,145.100367,0.901578,0.098422,True,False,True
1,Adams-Normandie,37987.0,1455.331389,3600.780013,5.886652,1621.694883,61.673873,32.257990,272.512175,11751.163222,1683.368756,310.656818,0.038311,0.094790,0.000155,0.042691,0.001624,0.000849,0.007174,0.309347,0.044314,0.008178,0.961689,True,False,False,False,False,False,False,False,False,37419.270130,5437.575802,1123.870291,4313.705512,0.206686,0.793314,False,True,False
2,Agoura Hills,27220.0,9336.980458,414.665537,0.000000,1054.881212,0.000000,32.058492,418.461988,1547.696671,1054.881212,450.520480,0.343019,0.015234,0.000000,0.038754,0.000000,0.001178,0.015373,0.056859,0.038754,0.016551,0.656981,True,False,False,False,False,False,False,False,False,121211.788628,4675.116219,3311.101230,1364.014989,0.708239,0.291761,True,False,True
3,Agua Dulce,17982.0,3430.950690,84.117674,29.477971,176.259175,2.276182,4.267842,130.656334,1025.422432,178.535357,164.402147,0.190799,0.004678,0.001639,0.009802,0.000127,0.000237,0.007266,0.057025,0.009929,0.009143,0.809201,True,False,False,False,False,False,False,False,False,108312.326136,1490.569409,1408.056410,82.512999,0.944643,0.055357,True,False,True
4,Alhambra,105896.0,3511.362136,809.817692,108.877418,19403.221726,191.097366,149.398101,773.197164,15428.325788,19594.319091,1031.472682,0.033159,0.007647,0.001028,0.183229,0.001805,0.001411,0.007301,0.145693,0.185034,0.009740,0.966841,True,False,False,False,False,False,False,False,False,61884.537960,13840.151455,5937.368580,7902.782874,0.428996,0.571004,False,True,False


#### Read LA neighborhoods geo file

In [21]:
la_hoods_gdf = gpd.read_file(
    "https://raw.githubusercontent.com/stiles/usc/main/data/geo/la-neighborhoods.geojson"
)

In [22]:
la_hoods_gdf.head()

,name,slug,county,type,city,region,geometry
0,Vermont Square,vermont-square,los-angeles,Los Angeles,los-angeles,south-la,"POLYGON ((-118.28273 34.01114, -118.28207 34.0..."
1,Arlington Heights,arlington-heights,los-angeles,Los Angeles,los-angeles,central-la,"POLYGON ((-118.31672 34.05250, -118.31649 34.0..."
2,Bel-Air,bel-air,los-angeles,Los Angeles,los-angeles,westside,"POLYGON ((-118.46076 34.13033, -118.46068 34.1..."
3,Beverly Crest,beverly-crest,los-angeles,Los Angeles,los-angeles,westside,"POLYGON ((-118.45139 34.13176, -118.45116 34.1..."
4,Beverlywood,beverlywood,los-angeles,Los Angeles,los-angeles,westside,"POLYGON ((-118.39426 34.05244, -118.39317 34.0..."


#### Merge the two

In [23]:
la_hoods_race_gdf = pd.merge(la_hoods_gdf, whites_la_df, on="name")

#### Export merged geo file

In [24]:
la_hoods_race_gdf.to_file("../data/geo/la_neighborhoods_race.geojson", driver="GeoJSON")